In [1]:
# !nvidia-smi

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
# 기본
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import gc

# 그래프 설정
sns.set()

# 그래프 기본 설정
plt.rcParams['font.family'] = 'Malgun Gothic'
# plt.rcParams['font.family'] = 'AppleGothic'
plt.rcParams['figure.figsize'] = 12, 6
plt.rcParams['font.size'] = 13
plt.rcParams['axes.unicode_minus'] = False

In [4]:
# 데이터프레임을 넣고 column별 특성 및 결측값, 고유값들을 확인하는 함수를 작성해본다.
# 필수는 아니지만 전체적인 흐름을 파악하기 쉬워진다.

def resumetable(df, n):
    print(f'데이터셋 크기: {df.shape}')                                # 데이터프레임의 전체 크기(행, 열) 출력

    summary = pd.DataFrame(df.dtypes, columns=['데이터 타입'])         # 각 피처의 데이터 타입을 가져와 데이터프레임으로 생성
    summary = summary.reset_index()                                    # 인덱스를 초기화하여 컬럼으로 변환
    summary = summary.rename(columns={'index':'피처'})                 # 'index' 컬럼명을 '피처'로 변경

    summary['결측값 개수'] = df.isnull().sum().values                 # 각 피처의 결측값(null) 개수 계산
    summary['고유값 개수'] = df.nunique().values                      # 각 피처의 고유값 개수 계산

    for i in range(n):
        summary[f"{i+1}번째 값"] = df.iloc[i].values

    return summary                                                     # 요약 테이블 반환

In [5]:
# 2. pandas로 데이터 불러오기
import pandas as pd

# ▶ Feature 파일 불러오기
df_features = pd.read_csv('/content/drive/MyDrive/멋쟁이사자처럼_파이널프로젝트/unique_1/train_1_회원정보_모델링용.csv')

In [6]:
# ▶ Segment 파일 불러오기 (ID + Segment)
df_segment = pd.read_csv('/content/drive/MyDrive/멋쟁이사자처럼_파이널프로젝트/unique_1/Segment_병합.csv')

# ▶ Segment만 순서대로 붙이기 (ID 순서 동일하다는 전제)
df_features['Segment'] = df_segment['Segment'].values

# ▶ 확인
display(resumetable(df_features,3))
df_features[['ID', 'Segment']]

데이터셋 크기: (2400000, 66)


,피처,데이터 타입,결측값 개수,고유값 개수,1번째 값,2번째 값,3번째 값
0,기준년월,int64,0,6,201807,201807,201807
1,ID,object,0,400000,TRAIN_000000,TRAIN_000001,TRAIN_000002
2,남녀구분코드,int64,0,2,2,1,1
3,회원여부_이용가능,int64,0,2,1,1,1
4,회원여부_이용가능_CA,int64,0,2,1,1,1
...,...,...,...,...,...,...,...
61,Life_Stage,object,0,7,자녀성장(2),자녀성장(1),자녀출산기
62,최종카드발급경과월,int64,0,68,22,18,20
63,연령_num,int64,0,6,45,35,35
64,연회비발생카드여부_B0M,int64,0,2,0,0,0


,ID,Segment
0,TRAIN_000000,D
1,TRAIN_000001,E
2,TRAIN_000002,C
3,TRAIN_000003,D
4,TRAIN_000004,E
...,...,...
2399995,TRAIN_399995,E
2399996,TRAIN_399996,D
2399997,TRAIN_399997,C
2399998,TRAIN_399998,E


In [7]:
from sklearn.preprocessing import LabelEncoder

# 1. Segment를 숫자로 변환
le = LabelEncoder()
df_features['Segment_encoded'] = le.fit_transform(df_features['Segment'])

# 2. 숫자형 컬럼만 선택
numeric_cols = df_features.select_dtypes(include=['number']).columns.drop('Segment_encoded', errors='ignore')

# 3. 상관계수 계산
correlations = df_features[numeric_cols].corrwith(df_features['Segment_encoded'])

# ✅ 절댓값 적용 여부 → 아래 한 줄 주석처리로 토글
correlations = correlations.abs()

# 4. 정렬
correlations_sorted = correlations.sort_values(ascending=False)

# 5. 출력 형식
pd.set_option('display.float_format', lambda x: f'{x:.6f}')
pd.set_option('display.max_rows', None)

# 6. 출력
display(correlations_sorted)

,0
이용금액_R3M_신용체크,0.622827
이용금액_R3M_신용,0.589032
_1순위카드이용금액,0.573870
_2순위카드이용금액,0.400225
이용카드수_신용체크,0.397713
_1순위카드이용건수,0.383609
이용카드수_신용,0.376918
_2순위카드이용건수,0.376494
이용가능카드수_신용,0.353014
이용가능카드수_신용체크,0.352211


In [8]:
# 7. 상관계수 0.35 이상 컬럼만 선택
selected_cols = correlations_sorted[correlations_sorted >= 0.35].index.tolist()

# 8. 해당 컬럼만 추출
X_selected = df_features[selected_cols]

# 9. 확인
print("선택된 컬럼 수:", len(selected_cols))
display(X_selected.head(10))

선택된 컬럼 수: 10


,이용금액_R3M_신용체크,이용금액_R3M_신용,_1순위카드이용금액,_2순위카드이용금액,이용카드수_신용체크,_1순위카드이용건수,이용카드수_신용,_2순위카드이용건수,이용가능카드수_신용,이용가능카드수_신용체크
0,196,196,3681,0,1,26,1,0,1,2
1,13475,13475,13323,0,1,46,1,0,1,1
2,23988,23988,24493,0,1,28,1,0,1,2
3,3904,3904,5933,0,1,1,1,0,2,3
4,1190,0,0,0,1,-2,0,0,1,2
5,0,0,0,0,0,1,0,0,1,1
6,10571,10571,10378,0,1,32,1,0,1,1
7,3463,3463,1062,3196,2,49,2,25,2,2
8,124967,121279,68078,38051,5,176,4,128,4,7
9,0,0,0,0,0,-2,0,0,0,1


In [9]:
from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.tools.tools import add_constant

def calculate_vif(df):
    X_const = add_constant(df)  # 상수항 추가
    vif_data = pd.DataFrame()
    vif_data["feature"] = df.columns
    vif_data["VIF"] = [
        variance_inflation_factor(X_const.values, i + 1)  # +1: 상수항 제외
        for i in range(df.shape[1])
    ]
    return vif_data.sort_values(by="VIF", ascending=False)

In [10]:
# 결측치 제거가 완료되었으므로 바로 계산
vif_result = calculate_vif(X_selected)

# 결과 확인
display(vif_result)

,feature,VIF
1,이용금액_R3M_신용,84.260303
2,_1순위카드이용금액,41.933094
3,_2순위카드이용금액,27.265139
6,이용카드수_신용,15.804013
7,_2순위카드이용건수,14.485353
0,이용금액_R3M_신용체크,14.107237
4,이용카드수_신용체크,13.225589
8,이용가능카드수_신용,6.093288
9,이용가능카드수_신용체크,3.527507
5,_1순위카드이용건수,3.112086


In [11]:
# import statsmodels.api as sm
# from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn.model_selection import cross_val_score
from xgboost import XGBClassifier
import warnings

warnings.filterwarnings('ignore')

# 교차검증 함수 수정
def evaluate_cv_score(X_input, y_input, cv=5):
    model = XGBClassifier(
        random_state=42,
        n_jobs=-1,
        use_label_encoder=False,
        eval_metric='mlogloss',
        tree_method='gpu_hist'  # GPU 사용 설정
    )
    scores = cross_val_score(model, X_input, y_input, cv=cv, scoring='f1_micro')
    return scores.mean()

# ▶ 초기 설정
X_vif = X_selected.copy()
y_vif = df_features.loc[X_vif.index, 'Segment_encoded']
iteration = 0

while True:
    # 1. 상수항 추가 후 VIF 계산
    X_const = sm.add_constant(X_vif)
    vif = pd.DataFrame()
    vif['변수'] = X_const.columns
    vif['VIF'] = [variance_inflation_factor(X_const.values, i) for i in range(X_const.shape[1])]
    vif = vif.sort_values(by='VIF', ascending=False)

    # 2. const는 제외하고 가장 VIF 높은 컬럼 찾기
    vif_no_const = vif[vif['변수'] != 'const']
    top_vif = vif_no_const.iloc[0]

    print(f"[{iteration}] 🔍 최상위 VIF 컬럼: {top_vif['변수']} (VIF={top_vif['VIF']:.2f})")

    # 3. 모두 기준 이하라면 종료
    if top_vif['VIF'] <= 10:
        print(f"✅ 모든 컬럼의 VIF가 10 이하입니다. 종료합니다.")
        break

    # 4. 성능 측정
    f1 = evaluate_cv_score(X_vif, y_vif)
    print(f"   현재 컬럼 수: {X_vif.shape[1]}, F1 Score: {f1:.4f}")

    # 5. 컬럼 제거
    print(f"   🗑️ '{top_vif['변수']}' 제거\n")
    X_vif = X_vif.drop(columns=[top_vif['변수']])
    iteration += 1


[0] 🔍 최상위 VIF 컬럼: 이용금액_R3M_신용 (VIF=84.26)
   현재 컬럼 수: 10, F1 Score: 0.8434
   🗑️ '이용금액_R3M_신용' 제거

[1] 🔍 최상위 VIF 컬럼: 이용카드수_신용 (VIF=15.59)
   현재 컬럼 수: 9, F1 Score: 0.8430
   🗑️ '이용카드수_신용' 제거

[2] 🔍 최상위 VIF 컬럼: _2순위카드이용금액 (VIF=14.66)
   현재 컬럼 수: 8, F1 Score: 0.8430
   🗑️ '_2순위카드이용금액' 제거

[3] 🔍 최상위 VIF 컬럼: 이용금액_R3M_신용체크 (VIF=9.31)
✅ 모든 컬럼의 VIF가 10 이하입니다. 종료합니다.


In [12]:
# 루프 종료 후 최종 컬럼 확인
print("✅ 최종 남은 컬럼 수:", len(X_vif.columns))
print("✅ 최종 남은 컬럼 목록:")
print(X_vif.columns.tolist())

✅ 최종 남은 컬럼 수: 7
✅ 최종 남은 컬럼 목록:
['이용금액_R3M_신용체크', '_1순위카드이용금액', '이용카드수_신용체크', '_1순위카드이용건수', '_2순위카드이용건수', '이용가능카드수_신용', '이용가능카드수_신용체크']
